# Sprint 7 · Webinar 21 · Data Analytics teórico (Valores atípicos, segmentación y GitHub)

**Programa:** Data Analytics  
**Sprint:** 7  
**Modalidad:** Teórica (con demostraciones en Python)  


## Fecha

Completa la información de la sesión:

- **Fecha:**  
- **Instructor/a:**  
- **Duración:** 100 minutos  


## Objetivos de la sesión teórica

Al finalizar esta sesión, la persona estudiante será capaz de:

1. **Identificar valores atípicos (outliers)** usando reglas estadísticas (IQR y Z-score).
2. **Interpretar outliers en contexto** (errores, casos de negocio válidos, eventos raros) y aplicar estrategias de tratamiento (verificación, recorte, transformación, segmentación).
3. **Crear segmentos de clientes** con sentencias `if/elif/else` y con `apply()` para reglas más flexibles.
4. **Redactar un Statistical Summary** breve y orientado a negocio con resultados reproducibles.
5. **Almacenar y compartir análisis** aplicando buenas prácticas de GitHub: repos, estructura de proyecto y workflow en Google Colab.


## Agenda sugerida (100 minutos)

| Tiempo | Bloque | Contenido | Modalidad |
|---:|---|---|---|
| 0–10 min | Calentamiento | ¿Qué es un outlier y por qué importa? | Discusión |
| 10–25 min | Dataset | Crear un dataset extenso y explorarlo | Demo + práctica guiada |
| 25–45 min | 7.3.1 | Identificar outliers con reglas estadísticas (IQR, Z-score) | Demo + preguntas |
| 45–55 min | 7.3.2 | Tratar outliers según el contexto (decisiones) | Discusión + demo |
| 55–70 min | 7.3.3 | Segmentación con sentencias `if` | Demo + mini-ejercicio |
| 70–80 min | 7.3.4 | Segmentación con `apply()` + funciones con `if` | Demo + mini-ejercicio |
| 80–88 min | 7.3.5 | Redacción de un Statistical Summary | Ejemplo guiado |
| 88–100 min | 7.4 | GitHub: conceptos, estructura y workflow con Colab | Guía práctica |


## Ejercicio 0 · Calentamiento en breakout rooms (discusión conceptual, 10 min)

**Objetivo:** Alinear criterios sobre qué es un valor atípico y cómo decidir si se “corrige” o se “interpreta”.

**Preguntas guía (discútanlas en equipo):**
1. Piensa en una métrica típica de negocio (ventas, visitas, tiempos, reclamos). ¿Qué podría verse como “outlier” y por qué?
2. ¿Un outlier siempre es “malo”? Da un ejemplo donde sea un **error** y otro donde sea un **caso real** (pero raro).
3. ¿Qué riesgos tiene eliminar outliers sin pensar en el contexto?
4. ¿Cómo documentarías una decisión de tratamiento de outliers para que otro analista la entienda?

**Salida esperada:** 3–5 bullet points con conclusiones del equipo.


## Ejercicio 1 · Crear y explorar un dataset extenso para análisis (15 min)

En esta sesión vamos a trabajar con un dataset **sintético** (generado por nosotros) que simula un escenario típico de analítica:

- **Clientes** con atributos demográficos y de adquisición.
- **Transacciones** (compras) con montos, descuentos y categorías.
- Una tabla de **métricas por cliente** (agregadas) para análisis de outliers y segmentación.

> En proyectos reales, el dataset vendría de una base de datos o un data warehouse. Aquí lo generamos para enfocarnos en los conceptos.

### 1.1 Generación del dataset

El siguiente código crea:
- `df_customers` (clientes)
- `df_transactions` (transacciones)
- `df_customer_metrics` (métricas agregadas por cliente)


In [ ]:
# ============================================================
# 1) Setup del entorno
# ============================================================
import numpy as np
import pandas as pd

# Para reproducibilidad: si todas las personas usan la misma semilla,
# obtendrán el mismo dataset y los mismos resultados.
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

pd.set_option('display.max_columns', 50)



In [ ]:
# ============================================================
# 2) Crear dataset sintético de clientes
# ============================================================
n_customers = 5000

customer_id = np.arange(1, n_customers + 1)

regions = ['Norte', 'Centro', 'Sur', 'Occidente', 'Oriente']
channels = ['Organic', 'Ads', 'Referral', 'Partners', 'Email']
plans = ['Basic', 'Standard', 'Premium']

df_customers = pd.DataFrame({
    'customer_id': customer_id,
    'region': rng.choice(regions, size=n_customers, p=[0.18, 0.25, 0.22, 0.18, 0.17]),
    'acquisition_channel': rng.choice(channels, size=n_customers, p=[0.35, 0.25, 0.15, 0.10, 0.15]),
    'plan': rng.choice(plans, size=n_customers, p=[0.55, 0.35, 0.10]),
})

# Edad: normalmente 18–70, pero insertaremos algunos valores atípicos (errores o casos extremos)
age = rng.normal(loc=38, scale=12, size=n_customers).round().astype(int)
age = np.clip(age, 16, 85)

# Insertar outliers de edad (pocos, para analizar)
outlier_idx = rng.choice(n_customers, size=10, replace=False)
age[outlier_idx[:5]] = rng.integers(1, 10, size=5)      # edades demasiado bajas (posible error)
age[outlier_idx[5:]] = rng.integers(95, 120, size=5)    # edades demasiado altas (posible error)

df_customers['age'] = age

# Fecha de registro (signup)
start_date = pd.Timestamp('2023-01-01')
end_date = pd.Timestamp('2025-12-01')
signup_days = rng.integers(0, (end_date - start_date).days, size=n_customers)
df_customers['signup_date'] = start_date + pd.to_timedelta(signup_days, unit='D')

df_customers.head()


In [ ]:
# ============================================================
# 3) Crear dataset sintético de transacciones
# ============================================================
# Generaremos transacciones por cliente con distribución sesgada (muchas compras pequeñas y pocas grandes)
n_transactions = 120_000

transaction_id = np.arange(1, n_transactions + 1)
customer_for_tx = rng.choice(df_customers['customer_id'], size=n_transactions, replace=True)

categories = ['Suscripción', 'Add-on', 'Soporte', 'Capacitación', 'Servicios Profesionales']
payment_methods = ['Tarjeta', 'Transferencia', 'PSE', 'Efectivo', 'Billetera']

# Monto base: distribución log-normal para simular "cola larga" en gasto
amount = rng.lognormal(mean=3.0, sigma=0.8, size=n_transactions)  # ~20–2000 en la mayoría de casos

# Ajuste por plan: Premium tiende a tener tickets más altos
plan_map = df_customers.set_index('customer_id')['plan'].to_dict()
plan_factor = np.array([{'Basic': 0.9, 'Standard': 1.0, 'Premium': 1.3}[plan_map[c]] for c in customer_for_tx])
amount = amount * plan_factor

# Descuento: la mayoría 0–20%, algunos más altos (promos)
discount_pct = rng.choice([0, 5, 10, 15, 20, 30, 40, 50], size=n_transactions,
                          p=[0.35, 0.12, 0.18, 0.13, 0.10, 0.07, 0.03, 0.02])

# Fecha de transacción
tx_start = pd.Timestamp('2024-01-01')
tx_end = pd.Timestamp('2025-12-31')
tx_days = rng.integers(0, (tx_end - tx_start).days + 1, size=n_transactions)
tx_date = tx_start + pd.to_timedelta(tx_days, unit='D')

df_transactions = pd.DataFrame({
    'transaction_id': transaction_id,
    'customer_id': customer_for_tx,
    'transaction_date': tx_date,
    'category': rng.choice(categories, size=n_transactions, p=[0.55, 0.15, 0.15, 0.08, 0.07]),
    'payment_method': rng.choice(payment_methods, size=n_transactions, p=[0.55, 0.18, 0.15, 0.05, 0.07]),
    'amount_gross': amount.round(2),
    'discount_pct': discount_pct
})

# Aplicar descuento: amount_net = amount_gross * (1 - discount_pct/100)
df_transactions['amount_net'] = (df_transactions['amount_gross'] * (1 - df_transactions['discount_pct'] / 100)).round(2)

# Insertar outliers en montos: algunas transacciones extremadamente altas (casos raros o errores)
high_outlier_rows = rng.choice(n_transactions, size=25, replace=False)
df_transactions.loc[high_outlier_rows, 'amount_gross'] = rng.uniform(20_000, 120_000, size=25).round(2)
df_transactions.loc[high_outlier_rows, 'amount_net'] = (df_transactions.loc[high_outlier_rows, 'amount_gross'] *
                                                       (1 - df_transactions.loc[high_outlier_rows, 'discount_pct'] / 100)).round(2)

# Insertar montos negativos: simulamos reembolsos (no necesariamente outliers, sino otro tipo de evento)
refund_rows = rng.choice(n_transactions, size=60, replace=False)
df_transactions.loc[refund_rows, 'amount_gross'] = -rng.uniform(10, 2000, size=60).round(2)
df_transactions.loc[refund_rows, 'amount_net'] = df_transactions.loc[refund_rows, 'amount_gross']

df_transactions.head()


In [ ]:
# ============================================================
# 4) Crear métricas por cliente (dataset de trabajo principal)
# ============================================================
# En analítica es muy común pasar de "eventos" (transacciones) a métricas agregadas por entidad (cliente).
# Esto facilita: outlier detection, segmentación, dashboards, etc.

# Fijamos una fecha de corte (como si fuera "hoy" para el análisis)
analysis_date = pd.Timestamp('2026-01-01')

tx = df_transactions.copy()

# Separar compras (monto positivo) de reembolsos (monto negativo)
tx['is_refund'] = tx['amount_net'] < 0

agg = tx.groupby('customer_id').agg(
    num_transactions=('transaction_id', 'count'),
    num_refunds=('is_refund', 'sum'),
    total_spend=('amount_net', 'sum'),
    avg_ticket=('amount_net', 'mean'),
    max_ticket=('amount_net', 'max'),
    last_tx_date=('transaction_date', 'max')
).reset_index()

agg['recency_days'] = (analysis_date - agg['last_tx_date']).dt.days
agg['refund_rate'] = (agg['num_refunds'] / agg['num_transactions']).round(4)

# Unir con atributos del cliente
df_customer_metrics = df_customers.merge(agg, on='customer_id', how='left')

# Clientes sin transacciones: rellenamos con ceros y recency alta
df_customer_metrics['num_transactions'] = df_customer_metrics['num_transactions'].fillna(0).astype(int)
df_customer_metrics['num_refunds'] = df_customer_metrics['num_refunds'].fillna(0).astype(int)
df_customer_metrics['total_spend'] = df_customer_metrics['total_spend'].fillna(0.0)
df_customer_metrics['avg_ticket'] = df_customer_metrics['avg_ticket'].fillna(0.0)
df_customer_metrics['max_ticket'] = df_customer_metrics['max_ticket'].fillna(0.0)
df_customer_metrics['last_tx_date'] = df_customer_metrics['last_tx_date'].fillna(pd.NaT)
df_customer_metrics['recency_days'] = df_customer_metrics['recency_days'].fillna((analysis_date - df_customer_metrics['signup_date']).dt.days).astype(int)
df_customer_metrics['refund_rate'] = df_customer_metrics['refund_rate'].fillna(0.0)

df_customer_metrics.head()


In [ ]:
# ============================================================
# 5) Exploración rápida del dataset
# ============================================================
df_customer_metrics.info()



In [ ]:
# Estadísticas descriptivas de algunas variables numéricas clave
df_customer_metrics[['age', 'num_transactions', 'total_spend', 'avg_ticket', 'max_ticket', 'recency_days', 'refund_rate']].describe().T


### Preguntas guiadas para el Ejercicio 1

1. ¿Qué columnas son **numéricas** y cuáles son **categóricas**?
2. ¿Hay clientes con `num_transactions = 0`? ¿Qué significa en un caso real?
3. Observa `total_spend`, `avg_ticket` y `max_ticket`. ¿Se intuye la presencia de valores extremos?
4. ¿Qué variable te parece más útil para medir “actividad reciente” del cliente?

> Guarda estas observaciones: te servirán para justificar decisiones en el Statistical Summary.


## 7.3 Análisis de valores atípicos y segmentación

### 7.3.1 Identificando valores atípicos con reglas estadísticas (20 min)

Un **valor atípico (outlier)** es una observación que se aleja significativamente del patrón general de los datos.

En analítica, los outliers suelen aparecer por:
- **Errores de captura/ETL** (un cero de más, campos invertidos, tipos mal parseados).
- **Eventos reales pero raros** (compras corporativas grandes, campañas, estacionalidad).
- **Cambios de definición** (nuevas reglas de negocio, ajustes contables).

Dos reglas comunes para detectarlos:
1. **IQR (Interquartile Range):** usa los percentiles 25% (Q1) y 75% (Q3).  
   - Se define: `IQR = Q3 - Q1`  
   - Outliers típicos: valores fuera de `[Q1 - 1.5*IQR, Q3 + 1.5*IQR]`
2. **Z-score:** mide cuántas desviaciones estándar se aleja un punto de la media.  
   - Se define: `z = (x - mean) / std`  
   - Umbrales típicos: `|z| > 3` (depende del contexto y la distribución)

**Nota práctica:** si la variable es muy asimétrica (por ejemplo, `total_spend`), IQR suele ser más robusto que Z-score.


In [ ]:
# ============================================================
# 7.3.1 A) Detección de outliers con IQR en total_spend
# ============================================================
metric = 'total_spend'

q1 = df_customer_metrics[metric].quantile(0.25)
q3 = df_customer_metrics[metric].quantile(0.75)
iqr = q3 - q1

lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

df_customer_metrics['outlier_iqr_total_spend'] = (
    (df_customer_metrics[metric] < lower_bound) |
    (df_customer_metrics[metric] > upper_bound)
)

outlier_count = df_customer_metrics['outlier_iqr_total_spend'].sum()
outlier_pct = outlier_count / len(df_customer_metrics)

print(f"IQR bounds para {metric}: [{lower_bound:,.2f}, {upper_bound:,.2f}]")
print(f"Outliers detectados (IQR): {outlier_count:,} ({outlier_pct:.2%})")

df_customer_metrics.loc[df_customer_metrics['outlier_iqr_total_spend'], 
                        ['customer_id','plan','region','num_transactions','total_spend','max_ticket']].head(10)


In [ ]:
# ============================================================
# 7.3.1 B) Detección de outliers con Z-score en total_spend
# ============================================================
# Importante: Z-score asume una distribución relativamente "normal".
# En variables con cola larga (como gasto), puede marcar demasiados outliers o no ser estable.

metric = 'total_spend'
mean = df_customer_metrics[metric].mean()
std = df_customer_metrics[metric].std(ddof=0)

# Para evitar división por cero
if std == 0:
    df_customer_metrics['z_total_spend'] = 0.0
else:
    df_customer_metrics['z_total_spend'] = (df_customer_metrics[metric] - mean) / std

df_customer_metrics['outlier_z_total_spend'] = df_customer_metrics['z_total_spend'].abs() > 3

outlier_count_z = df_customer_metrics['outlier_z_total_spend'].sum()
outlier_pct_z = outlier_count_z / len(df_customer_metrics)

print(f"Outliers detectados (Z-score |z|>3): {outlier_count_z:,} ({outlier_pct_z:.2%})")

df_customer_metrics.loc[df_customer_metrics['outlier_z_total_spend'],
                        ['customer_id','plan','num_transactions','total_spend','z_total_spend']].head(10)


In [ ]:
# ============================================================
# 7.3.1 C) Visualización rápida (distribución + valores extremos)
# ============================================================
import matplotlib.pyplot as plt

# Histograma con límites razonables para ver la masa principal
plt.figure(figsize=(10, 4))
plt.hist(df_customer_metrics['total_spend'], bins=60)
plt.title('Distribución de total_spend (incluye cola larga)')
plt.xlabel('total_spend')
plt.ylabel('frecuencia')
plt.show()

# Boxplot ayuda a ver "cola" y outliers (aunque comprime valores muy grandes)
plt.figure(figsize=(10, 2.5))
plt.boxplot(df_customer_metrics['total_spend'], vert=False)
plt.title('Boxplot de total_spend')
plt.xlabel('total_spend')
plt.show()


### Mini-ejercicio (5 min)

1. Repite el análisis IQR para `max_ticket` y responde:
   - ¿Cuántos outliers detectas?
   - ¿Tiene sentido que existan tickets máximos muy altos?

2. Repite el Z-score para `age` y revisa si detecta las edades extremas.

> Tip: duplica el bloque de IQR y cambia `metric = 'max_ticket'`.


### 7.3.2 Cómo abordar valores atípicos según el contexto (10 min)

Detectar outliers es solo el primer paso. La decisión correcta depende del **contexto**:

**Preguntas de negocio antes de actuar**
- ¿El outlier es un **error** (captura/ETL) o un **evento real**?
- ¿La métrica representa un **proceso diferente**? (ej.: reembolsos vs ventas)
- ¿El outlier impacta el resultado del análisis? (medias, modelos, segmentación)

**Estrategias comunes**
1. **Validar y corregir** (si es error y hay fuente confiable).
2. **Eliminar** (si es error sin corrección posible, y documentando impacto).
3. **Capping / Winsorization**: recortar valores a percentiles (p. ej., 1% y 99%).
4. **Transformación**: log, raíz, escalamiento robusto (útil para variables sesgadas).
5. **Segmentar**: tratar los “casos especiales” como un grupo distinto (p. ej., cuentas corporativas).

A continuación veremos ejemplos prácticos para `total_spend` y el caso especial de reembolsos.


In [ ]:
# ============================================================
# 7.3.2 A) Separar reembolsos del gasto total (contexto)
# ============================================================
# Nota: total_spend ya incluye negativos porque sumamos amount_net.
# Si queremos "gasto bruto" sin reembolsos, podemos calcularlo aparte.

tx_pos = df_transactions[df_transactions['amount_net'] > 0].copy()
tx_neg = df_transactions[df_transactions['amount_net'] < 0].copy()

spend_pos = tx_pos.groupby('customer_id')['amount_net'].sum().rename('spend_positive')
spend_neg = tx_neg.groupby('customer_id')['amount_net'].sum().rename('refund_amount')  # negativo

df_customer_metrics = df_customer_metrics.merge(spend_pos, on='customer_id', how='left')
df_customer_metrics = df_customer_metrics.merge(spend_neg, on='customer_id', how='left')

df_customer_metrics['spend_positive'] = df_customer_metrics['spend_positive'].fillna(0.0)
df_customer_metrics['refund_amount'] = df_customer_metrics['refund_amount'].fillna(0.0)

# Métrica alternativa: net_spend = spend_positive + refund_amount (refund_amount es negativo)
df_customer_metrics['net_spend'] = df_customer_metrics['spend_positive'] + df_customer_metrics['refund_amount']

df_customer_metrics[['customer_id','spend_positive','refund_amount','net_spend','refund_rate']].head()


In [ ]:
# ============================================================
# 7.3.2 B) Capping (recorte) usando percentil 99 para net_spend
# ============================================================
metric = 'net_spend'
p99 = df_customer_metrics[metric].quantile(0.99)

df_customer_metrics['net_spend_capped_p99'] = df_customer_metrics[metric].clip(upper=p99)

print(f"Percentil 99 de {metric}: {p99:,.2f}")
df_customer_metrics[[metric, 'net_spend_capped_p99']].describe().T


In [ ]:
# ============================================================
# 7.3.2 C) Transformación logarítmica (para cola larga)
# ============================================================
# log1p(x) = log(1+x) evita problemas con x=0.
# Nota: si hay valores negativos, primero definimos una métrica no-negativa.
# Aquí usamos spend_positive (solo compras).

df_customer_metrics['log_spend_positive'] = np.log1p(df_customer_metrics['spend_positive'])

df_customer_metrics[['spend_positive', 'log_spend_positive']].describe().T


### Pregunta clave

Si estás construyendo un **segmento “VIP”** basado en gasto, ¿es mejor usar:
- `total_spend` (incluye reembolsos),
- `spend_positive` (solo compras),
- `net_spend` (compras + reembolsos),
- o `net_spend_capped_p99` (recortado)?

No hay una única respuesta. Debes justificarla con el objetivo del análisis.


### 7.3.3 Segmentación de clientes con sentencias if (15 min)

La **segmentación** consiste en agrupar clientes con características similares para:
- Analizar comportamiento (retención, churn, monetización).
- Definir estrategias (promociones, atención, pricing).
- Comparar desempeño por grupo.

En Python, una forma didáctica de segmentar es con **sentencias `if/elif/else`**.

En este ejemplo crearemos un segmento simple tipo **RFM-lite** (sin puntajes):
- **Recency (R):** días desde la última compra (`recency_days`)
- **Monetary (M):** gasto neto (`net_spend`)

Reglas (ejemplo):
- `VIP`: gasto alto y compra reciente
- `Leal`: compra reciente con gasto medio
- `En riesgo`: hace mucho que no compra, pero antes gastaba
- `Nuevo / Inactivo`: poco gasto y poca actividad


In [ ]:
# ============================================================
# 7.3.3 A) Función con if/elif/else para segmentación
# ============================================================
def segment_customer(recency_days: int, net_spend: float) -> str:
    """Asigna un segmento basado en recencia y gasto.

    Parámetros
    ----------
    recency_days : int
        Días desde la última transacción (menor = más reciente).
    net_spend : float
        Gasto neto total (compras + reembolsos).

    Retorna
    -------
    str
        Nombre del segmento.
    """
    if (net_spend >= 2000) and (recency_days <= 30):
        return 'VIP'
    elif (net_spend >= 800) and (recency_days <= 60):
        return 'Leal'
    elif (net_spend >= 800) and (recency_days > 60):
        return 'En riesgo'
    else:
        return 'Nuevo / Inactivo'

# Aplicar la función a dos columnas (vectorizando con apply sobre filas es simple de entender,
# aunque para grandes volúmenes puede ser más lento que alternativas vectorizadas).
df_customer_metrics['segment_if'] = df_customer_metrics.apply(
    lambda row: segment_customer(row['recency_days'], row['net_spend']),
    axis=1
)

df_customer_metrics['segment_if'].value_counts(dropna=False)


In [ ]:
# Ver algunos ejemplos por segmento
(df_customer_metrics[['customer_id','plan','recency_days','net_spend','segment_if']]
 .sort_values(['segment_if','net_spend'], ascending=[True, False])
 .head(15))


### Mini-ejercicio (5 min)

Ajusta los umbrales (por ejemplo, el gasto de VIP o la ventana de recencia) y observa:
- Cómo cambia el tamaño de cada segmento.
- Si el segmento VIP queda “demasiado grande” o “demasiado pequeño”.

**Regla práctica:** los umbrales deberían tener sentido para tu caso de negocio y ser estables en el tiempo.


### 7.3.4 Segmentación usando IF y `apply()` con funciones que crean nuevas columnas (10 min)

A veces una segmentación requiere múltiples salidas:
- `segment` (etiqueta)
- `risk_flag` (bandera de riesgo)
- `recommended_action` (acción sugerida)

Podemos encapsular esa lógica en una función y devolver un `pd.Series` para crear **varias columnas** con `apply()`.


In [ ]:
# ============================================================
# 7.3.4 A) Función que retorna múltiples columnas
# ============================================================
def segment_and_action(recency_days: int, net_spend: float, refund_rate: float) -> pd.Series:
    """Devuelve segmento, bandera de riesgo y acción sugerida.

    La idea no es que estas reglas sean 'perfectas', sino mostrar cómo:
    - integrar IFs,
    - manejar casos especiales,
    - crear múltiples columnas de salida.
    """
    # 1) Caso especial: alta tasa de reembolsos
    if refund_rate >= 0.20 and net_spend > 0:
        return pd.Series({
            'segment_v2': 'Reembolsos altos',
            'risk_flag': 'ALTO',
            'recommended_action': 'Revisar políticas / fraude / experiencia'
        })

    # 2) Segmentación principal por recencia + gasto
    if (net_spend >= 2000) and (recency_days <= 30):
        return pd.Series({'segment_v2': 'VIP', 'risk_flag': 'BAJO', 'recommended_action': 'Upsell + atención prioritaria'})
    elif (net_spend >= 800) and (recency_days <= 60):
        return pd.Series({'segment_v2': 'Leal', 'risk_flag': 'BAJO', 'recommended_action': 'Cross-sell + fidelización'})
    elif (net_spend >= 800) and (recency_days > 60):
        return pd.Series({'segment_v2': 'En riesgo', 'risk_flag': 'MEDIO', 'recommended_action': 'Campaña de reactivación'})
    else:
        # Nota: net_spend puede ser 0 si nunca compró
        if recency_days <= 30:
            return pd.Series({'segment_v2': 'Nuevo', 'risk_flag': 'MEDIO', 'recommended_action': 'Onboarding / activación'})
        return pd.Series({'segment_v2': 'Inactivo', 'risk_flag': 'ALTO', 'recommended_action': 'Winback o limpieza de CRM'})

# Aplicación: axis=1 para leer cada fila (didáctico).
new_cols = df_customer_metrics.apply(
    lambda row: segment_and_action(row['recency_days'], row['net_spend'], row['refund_rate']),
    axis=1
)

df_customer_metrics = pd.concat([df_customer_metrics, new_cols], axis=1)

df_customer_metrics[['segment_v2','risk_flag','recommended_action']].value_counts().head(10)


In [ ]:
# Distribución por segmento_v2 (tabla de conteos + porcentaje)
seg_dist = (df_customer_metrics['segment_v2']
            .value_counts()
            .rename_axis('segment_v2')
            .reset_index(name='count'))
seg_dist['pct'] = seg_dist['count'] / seg_dist['count'].sum()
seg_dist


### Nota técnica (para cuando avances)

`apply(axis=1)` es muy claro para aprender, pero puede ser más lento en datasets grandes.  
Alternativas comunes para producción:
- `np.select()` (vectorizado)
- `pd.cut()` (bins)
- reglas SQL en el DW/BI
- modelos (clustering, scoring, etc.)

Hoy priorizamos claridad conceptual y buenas prácticas de lectura.


### 7.3.5 Redacción de un Statistical Summary (8 min)

Un **Statistical Summary** es un texto corto (1–2 páginas, a veces menos) que resume:

1. **Objetivo del análisis** (pregunta de negocio).
2. **Datos usados** (fuente, periodo, tamaño, variables relevantes).
3. **Métodos** (reglas de outliers, segmentación, métricas).
4. **Hallazgos clave** (números concretos, comparaciones, insights).
5. **Limitaciones** (supuestos, sesgos, calidad de datos).
6. **Recomendaciones / próximos pasos**.

El propósito es que una persona no técnica pueda entender:
- qué hiciste,
- qué encontraste,
- y qué debería hacerse después.

A continuación generamos algunos números que suelen alimentar ese resumen.


In [ ]:
# ============================================================
# 7.3.5 A) KPIs de contexto para el summary
# ============================================================
from IPython.display import display
n = len(df_customer_metrics)

# Outliers (ejemplo): según IQR en total_spend
out_iqr = df_customer_metrics['outlier_iqr_total_spend'].sum()
out_iqr_pct = out_iqr / n

# Segmentos
seg_counts = df_customer_metrics['segment_v2'].value_counts()
vip_count = seg_counts.get('VIP', 0)
risk_high = (df_customer_metrics['risk_flag'] == 'ALTO').sum()

# Métricas descriptivas (usando spend_positive para evitar negativos)
spend_desc = df_customer_metrics['spend_positive'].describe()

print(f"Clientes: {n:,}")
print(f"Outliers IQR (total_spend): {out_iqr:,} ({out_iqr_pct:.2%})")
print(f"VIP (segment_v2): {vip_count:,} ({vip_count/n:.2%})")
print(f"Riesgo ALTO: {risk_high:,} ({risk_high/n:.2%})")
print('\nDescriptivos spend_positive:')
display(spend_desc)


#### Ejemplo de Statistical Summary (plantilla)

Completa los campos con tus números (puedes copiar/pegar y editar):

**Objetivo:**  
Analizar patrones de gasto y actividad de clientes, identificar valores atípicos y proponer segmentos accionables para estrategias de retención y monetización.

**Datos:**  
- Base sintética de clientes: `n = {n:,}`  
- Periodo de transacciones: 2024-01-01 a 2025-12-31  
- Variables clave: `net_spend`, `spend_positive`, `recency_days`, `refund_rate`, `plan`, `acquisition_channel`, `region`.

**Método:**  
- Outliers detectados con IQR sobre `total_spend`.  
- Tratamiento: separación de reembolsos + capping p99 para robustez en análisis agregados.  
- Segmentación por reglas (recencia + gasto) con casos especiales de reembolsos altos.

**Hallazgos clave (ejemplo):**
- Se detectaron **{out_iqr:,} outliers** en `total_spend` (**{out_iqr_pct:.2%}**).  
- El segmento **VIP** representa **{vip_count:,} clientes** (**{vip_count/n:.2%}**).  
- Clientes con **riesgo ALTO**: **{risk_high:,}** (**{risk_high/n:.2%}**).  
- Recomendación: priorizar campañas de reactivación para “En riesgo” y revisar causas para “Reembolsos altos”.

**Limitaciones:**  
- Dataset sintético; en producción validar con fuentes reales y reglas contables.  
- Los umbrales de segmentación deben calibrarse con criterios de negocio y estacionalidad.

**Próximos pasos:**  
- Validar outliers con equipo de negocio (operaciones/finanzas).  
- Ajustar umbrales por plan/región.  
- Construir dashboard con distribución de segmentos y evolución mensual.


## 7.4 Almacenando y compartiendo análisis (12 min)

En analítica profesional, no basta con “tener un notebook que funciona”:
- necesitas **reproducibilidad**, **colaboración** y **trazabilidad**.

GitHub (Git) es el estándar para lograrlo.

### 7.4.1 GitHub para analistas: conceptos e importancia

**Conceptos clave**
- **Repositorio (repo):** carpeta versionada con historial.
- **Commit:** “foto” del estado del proyecto con mensaje.
- **Branch:** línea de trabajo paralela (p. ej., `main`, `feature/outliers`).
- **Pull Request (PR):** revisión e integración de cambios.
- **Issues:** tareas y seguimiento.

**Valor para analistas**
- Historial de cambios y decisiones (auditable).
- Colaboración con revisiones (PR).
- Estandariza estructura y documentación (README).
- Facilita despliegues y automatización (CI/CD).


### 7.4.2 Repos & estructura de proyectos (configuración inicial)

Una estructura simple y útil para análisis:

```
mi-proyecto-analitica/
├─ data/                # datos (ideal: no subir datos sensibles)
│  ├─ raw/              # datos crudos (solo lectura)
│  └─ processed/        # datos listos para análisis
├─ notebooks/           # notebooks exploratorios
├─ src/                 # funciones reutilizables (python)
├─ reports/             # resultados: tablas, figuras, pdf
├─ README.md            # qué es, cómo correrlo
├─ requirements.txt     # dependencias
└─ .gitignore           # excluir archivos (data pesada, credenciales)
```

**Buenas prácticas mínimas**
- Los notebooks van a `notebooks/`.
- Lo reusable va a `src/` (funciones para outliers, segmentación).
- Documenta en `README.md` cómo reproducir.
- Usa `.gitignore` para excluir datos sensibles, caches, credenciales.


In [ ]:
# ============================================================
# (Demo opcional) Crear una estructura de proyecto local
# ============================================================
# En Colab o en tu máquina puedes crear la estructura base.
# Este bloque NO sube nada a GitHub; solo crea carpetas.
import os

project_root = 'mi-proyecto-analitica'
folders = [
    f'{project_root}/data/raw',
    f'{project_root}/data/processed',
    f'{project_root}/notebooks',
    f'{project_root}/src',
    f'{project_root}/reports'
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

# Crear archivos básicos
readme_path = f'{project_root}/README.md'
gitignore_path = f'{project_root}/.gitignore'
requirements_path = f'{project_root}/requirements.txt'

if not os.path.exists(readme_path):
    with open(readme_path, 'w', encoding='utf-8') as f:
        f.write('# Mi proyecto de analítica\n\nDescripción corta del objetivo del análisis.\n')

if not os.path.exists(gitignore_path):
    with open(gitignore_path, 'w', encoding='utf-8') as f:
        f.write('data/raw/\ndata/processed/\n__pycache__/\n.ipynb_checkpoints/\n.env\n')

if not os.path.exists(requirements_path):
    with open(requirements_path, 'w', encoding='utf-8') as f:
        f.write('pandas\nnumpy\nmatplotlib\n')

print('Estructura creada en:', project_root)


### 7.4.3 Aplicando GitHub Workflow con Google Colab

**Objetivo:** editar un notebook en Colab y versionarlo en GitHub con un workflow básico.

**Pasos típicos (resumen):**
1. Crear repo en GitHub: `mi-proyecto-analitica`.
2. Abrir Google Colab → montar Drive (opcional).
3. Clonar el repo en Colab:
   - `!git clone <URL_DEL_REPO>`
4. Crear una rama de trabajo:
   - `!git checkout -b feature/segmentacion`
5. Realizar cambios (notebook, README, scripts).
6. Guardar, agregar y hacer commit:
   - `!git status`
   - `!git add .`
   - `!git commit -m "Agregar segmentación v2 y resumen estadístico"`
7. Subir la rama a GitHub:
   - `!git push origin feature/segmentacion`
8. Abrir Pull Request en GitHub y solicitar revisión.

**Consejo:** en equipos, evita hacer cambios directos en `main`. Trabaja por ramas y PRs.


## 6. Take aways de la sesión teórica

Al terminar esta sesión, deberías llevarte estas ideas clave:

- Un outlier es un **síntoma**: puede ser error, evento raro o señal de negocio.
- IQR y Z-score son reglas útiles, pero **no sustituyen** el criterio y el contexto.
- Tratar outliers implica decisiones: validar, eliminar, recortar, transformar o segmentar.
- Segmentación con reglas (IF, apply) es una base práctica para análisis accionables.
- Un Statistical Summary transforma código en **comunicación de negocio**.
- GitHub permite reproducibilidad, colaboración y trazabilidad: “sin control de versiones, el análisis no es profesional”.


## 7. Cierre y próximos pasos

Para seguir profundizando:

1. Implementa la versión vectorizada de segmentación con `np.select()` y compárala con `apply()`.
2. Crea un pequeño dashboard (matplotlib) con:
   - distribución de segmentos,
   - gasto promedio por segmento,
   - recencia promedio por segmento.
3. Monta un repo en GitHub con la estructura propuesta y sube:
   - este notebook,
   - un `README.md`,
   - un `.gitignore`.


## 8. Información complementaria y recursos

- Documentación de pandas (apply, groupby, describe):  
  <https://pandas.pydata.org/docs/>
- Outliers y estadística robusta (conceptos):  
  - IQR (Tukey’s fences)  
  - Z-score (normalización)
- Git y GitHub:
  - Git Book (documentación oficial): <https://git-scm.com/book/en/v2>
  - GitHub Docs: <https://docs.github.com/>
- Google Colab + Git:
  - Guías de “clone / commit / push” en Colab (varían por credenciales/entorno)


## Cierre
**Kahoot de repaso (5 min)**
- Profundizamos en la detección de anomalías y segmentación.
- Vimos cómo usar reglas estadísticas para flaggear transacciones sospechosas.

**Reflexión:**
- ¿Es lo mismo un outlier que un error de datos? ¿Por qué?
- ¿Cómo aporta valor al negocio segmentar a los clientes?

**Q&A y próximos pasos.**


## Siguientes Pasos
- **Próxima sesión:** Sprint 7 - Proyecto final del sprint: Clínica Agendada.
- **Participación:** Revisa los conceptos de media, mediana y desviación estándar.
- **Recordatorios:** Prepara tu entorno para el proyecto integrado.
